In [26]:
import pandas as pd 
import os
from os import listdir
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import torch
import torchvision.transforms as transforms
import numpy as np
import torchvision
import torchvision
from PIL import Image
import os
from tqdm import tqdm

scaled_train_dir = "B:\\CV Drone Detection\\scaled\\train\\drone"
scaled_test_dir = "B:\\CV Drone Detection\\scaled\\test\\drone"
scaled_valid_dir = "B:\\CV Drone Detection\\scaled\\valid\\drone"

In [28]:
def lensize_img(path):
    img_list = []
    img_res = {}
    for img in os.listdir(path):
        if img.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(path, img)
            img_list.append(img)
            if Image.open(img_path).size in img_res:
                img_res[Image.open(img_path).size] += 1 
            else: 
                img_res[Image.open(img_path).size] = 1

    return len(img_list), img_res

In [29]:
birds_dir = "B:\\CV Drone Detection\\analysis_scaling\\cifar_filtered\\bird"
plain_dir = "B:\\CV Drone Detection\\analysis_scaling\\cifar_filtered\\airplane"

In [31]:
imgs_size_b, img_res = lensize_img(birds_dir)
img_res
imgs_size_p, img_res = lensize_img(plain_dir)
img_res

{(32, 32): 6000}

In [43]:
false_data_size = imgs_size_b + imgs_size_p
false_data_size

12000

In [48]:
train_chunk = int(false_data_size * 0.7)
val_chunk = int(false_data_size * 0.2)
test_chunk = false_data_size - (train_chunk + val_chunk)

train_chunk, val_chunk, test_chunk, train_chunk + val_chunk + test_chunk

(8400, 2400, 1200, 12000)

In [20]:
lovely_dime = [128, 128]

In [ ]:
def scaler(in_dir, out_dir, lovely_dime=lovely_dime):
    os.makedirs(out_dir, exist_ok=True)
    processed = []
    
    for img in os.listdir(in_dir):
        if img.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(in_dir, img)
            image = cv2.imread(img_path)
            
            
            height, width = image.shape[:2]
            
            scale_factor_W, scale_factor_H = lovely_dime[0], lovely_dime[1]
            scale = min(scale_factor_W / width, scale_factor_H / height)

            new_width = int(width * scale)
            new_height = int(height * scale)    
            
            scaled_image = cv2.resize(src= image, 
                                    dsize =(new_width, new_height), 
                                    interpolation=cv2.INTER_CUBIC)
            
            canvas = np.zeros((scale_factor_H, scale_factor_W, 3), dtype=np.uint8)

            x_offset = (scale_factor_W - new_width) // 2
            y_offset = (scale_factor_H - new_height) // 2
            canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = scaled_image
            
            save_path = os.path.join(out_dir, img)
            cv2.imwrite(save_path, canvas)
            processed.append(save_path)
    return processed

In [22]:
out_dir = "B:\\CV Drone Detection\\scaled\\test\\notAdrone"

In [25]:
processed = scaler(in_dir=plain_dir, out_dir=out_dir)